# Voting Classifiers
---

## Introduction

To conclude with our attempts to try tackling the current problem, we are going to mobilize **voting classifiers**, aggregating a chosen selection of the previous models we have built.

As always, the prerequisite step consists on loading the appropriate packages to perform our work:

In [1]:
# Activate 'airbnb' environment:
!source activate airbnb

In [2]:
# Needed packages:
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.ensemble import (RandomForestClassifier,
                              GradientBoostingClassifier,
                              VotingClassifier)
from sklearn.neural_network import MLPClassifier
from utils import (create_training_testing_datasets,
                   calculate_dcg,
                   calculate_ndcg,
                   clf_prediction,
                   ndcg_mean_score_calc,
                   detailed_ndcg_mean_score_calc)

/Applications/anaconda3/envs/airbnb/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


---

## Create training and testing datasets

In [3]:
# Load the data:
consolidated_dataset = pd.read_csv("../data/consolidated_dataset.csv")

# Check basic info:
print("*** Some basic info:")
print("'consolidated_dataset' has {} data points with {} variables each.".format(*consolidated_dataset.shape))
print("'consolidated_dataset' counts {} missing values.".format(consolidated_dataset.isnull().sum().sum()))

# Give a look to the first lines:
print("\n*** First lines:")
display(consolidated_dataset.head())

*** Some basic info:
'consolidated_dataset' has 213451 data points with 161 variables each.
'consolidated_dataset' counts 0 missing values.

*** First lines:


,age,country_destination,nans,day_account_created,weekday_account_created,week_account_created,month_account_created,year_account_created,day_first_active,weekday_first_active,...,first_browser_SeaMonkey,first_browser_Silk,first_browser_SiteKiosk,first_browser_SlimBrowser,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_Yandex.Browser,first_browser_wOSBrowser
0,-1.0,NDF,1.225078,28,0,26,6,2010,19,3,...,0,0,0,0,0,0,0,0,0,0
1,38.0,NDF,-0.453135,25,2,21,5,2011,23,5,...,0,0,0,0,0,0,0,0,0,0
2,56.0,US,-0.453135,28,1,39,9,2010,9,1,...,0,0,0,0,0,0,0,0,0,0
3,42.0,other,-0.453135,5,0,49,12,2011,31,5,...,0,0,0,0,0,0,0,0,0,0
4,41.0,US,0.385972,14,1,37,9,2010,8,1,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Create training and testing datasets:
X_train, X_test, y_train, y_test, encoding_dict = create_training_testing_datasets(consolidated_dataset)

---

## Calculate Normalized DCG scores: "Out-of-the-box" voting classifier

In [5]:
# Calculate weight of each class:
class_weight = {x: 0 for x in range(12)}
for country_dest in y_train:
    class_weight[country_dest] += 1

# Initialize the classifiers in their "optimized" forms:

rf_clf = RandomForestClassifier(random_state=42,
                                n_estimators=77,
                                min_samples_split=10,
                                min_samples_leaf=1,
                                max_features='sqrt',
                                max_depth=None,
                                class_weight=class_weight,
                                bootstrap=False)

gb_clf = GradientBoostingClassifier(random_state=42)

mlp_clf = MLPClassifier(hidden_layer_sizes=(80, 40, 20),
                        activation='relu',
                        solver='adam',
                        learning_rate_init=0.0001,
                        max_iter=200,
                        random_state=42,
                        early_stopping=True,
                        validation_fraction=0.2,
                        n_iter_no_change=10)

# Initialize the voting classifier:
voting_clf = VotingClassifier(estimators=[('rf', rf_clf),
                                          ('gb', gb_clf),
                                          ('mlp', mlp_clf)],
                              voting='soft')

# Train the classifier:
print("Time info about classifier training:")
%time ootb_voting_clf = voting_clf.fit(X_train, y_train)

Time info about classifier training:
CPU times: user 39min 35s, sys: 45.1 s, total: 40min 21s
Wall time: 34min 7s


In [6]:
# Perform one prediction to check classifier:
print("Time info about classifier prediction:")
%time preds_list = clf_prediction(ootb_voting_clf, X_train[0])
print("***")

# Reverse encoding dictionary:
decoding_dict = dict(map(reversed, encoding_dict.items()))

# Print result:
print("For the classifier check:")
print("- Real first booking destination country: {}".format(decoding_dict[y_train[0]]))
print("- Predictions list: {}".format([decoding_dict[x] for x in preds_list]))

Time info about classifier prediction:
CPU times: user 128 ms, sys: 11.1 ms, total: 139 ms
Wall time: 122 ms
***
For the classifier check:
- Real first booking destination country: US
- Predictions list: ['US', 'NDF', 'other', 'FR', 'ES', 'IT', 'GB', 'CA', 'DE', 'NL', 'AU', 'PT']


In [7]:
# Calculate nDCG mean score on training dataset:
print("Time info about nDCG mean score calculation on training dataset:")
%time ndcg_mean_score = ndcg_mean_score_calc(ootb_voting_clf, X_train, y_train)
print("***")
print("On training dataset, classifier nDCG mean score is {:.6f}.".format(ndcg_mean_score))

Time info about nDCG mean score calculation on training dataset:
CPU times: user 44min 31s, sys: 50.4 s, total: 45min 21s
Wall time: 22min 41s
***
On training dataset, classifier nDCG mean score is 0.869396.


In [8]:
# Calculate nDCG mean score on testing dataset:
print("Time info about nDCG mean score calculation on testing dataset:")
%time ndcg_mean_score = ndcg_mean_score_calc(ootb_voting_clf, X_test, y_test)
print("***")
print("On testing dataset, classifier nDCG mean score is {:.6f}.".format(ndcg_mean_score))

Time info about nDCG mean score calculation on testing dataset:
CPU times: user 11min 8s, sys: 12.4 s, total: 11min 20s
Wall time: 5min 40s
***
On testing dataset, classifier nDCG mean score is 0.823524.


In [9]:
# Calculate nDCG mean score for each class on testing dataset:
print("Time info about nDCG mean score calculation for each class on testing dataset:")
%time ndcg_mean_scores_list = detailed_ndcg_mean_score_calc(ootb_voting_clf, X_test, y_test, encoding_dict)
print("***")
print("Detailed results for each class on testing dataset:")
for country_dest in range(12):
    print("nDCG mean score for {}: {:.6f}".format(decoding_dict[country_dest],
                                                  ndcg_mean_scores_list[country_dest]))

Time info about nDCG mean score calculation for each class on testing dataset:
CPU times: user 11min 8s, sys: 12.3 s, total: 11min 20s
Wall time: 5min 40s
***
Detailed results for each class on testing dataset:
nDCG mean score for AU: 0.000000
nDCG mean score for CA: 0.006763
nDCG mean score for DE: 0.007919
nDCG mean score for ES: 0.328300
nDCG mean score for FR: 0.427147
nDCG mean score for GB: 0.018735
nDCG mean score for IT: 0.054544
nDCG mean score for NDF: 0.948974
nDCG mean score for NL: 0.003289
nDCG mean score for PT: 0.000000
nDCG mean score for US: 0.793151
nDCG mean score for other: 0.497145


In [10]:
# Save model:
joblib.dump(ootb_voting_clf, "../models/ootb_voting_clf.pkl")

['../models/ootb_voting_clf.pkl']

---

## Conclusion

In this notebook, we are not going to try to obtain an "optimized" voting classifier, like we have done in the previous notebooks: Indeed, this process is very-long time consuming, it needs good hardware and calculation capacities.

In this current project, we can't benefit from these conditions.

As a quick conclusion, for the results obtained for the last "optimized" random forest classifier, we can note 4 major elements:
* On testing dataset, it gets a better nDCG mean score than the one obtained by the naive model.
* On testing dataset, it gets better nDCG mean scores for predicting correctly Canada, Germany, Spain, Great Britain, Netherlands and USA than the ones obtained by the naive model.
* On testing dataset, it gets worse nDCG mean scores for predicting correctly France, Italy, no destination found and other than the ones obtained by the naive model.
* On testing dataset, it is as "bad" as the naive model for predicting correctly Australia and Portugal.